In [1]:
import torch

In [2]:
words = open('names.txt','r').read().splitlines()

In [3]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [7]:
stoi, itos

({'a': 1,
  'b': 2,
  'c': 3,
  'd': 4,
  'e': 5,
  'f': 6,
  'g': 7,
  'h': 8,
  'i': 9,
  'j': 10,
  'k': 11,
  'l': 12,
  'm': 13,
  'n': 14,
  'o': 15,
  'p': 16,
  'q': 17,
  'r': 18,
  's': 19,
  't': 20,
  'u': 21,
  'v': 22,
  'w': 23,
  'x': 24,
  'y': 25,
  'z': 26,
  '.': 0},
 {1: 'a',
  2: 'b',
  3: 'c',
  4: 'd',
  5: 'e',
  6: 'f',
  7: 'g',
  8: 'h',
  9: 'i',
  10: 'j',
  11: 'k',
  12: 'l',
  13: 'm',
  14: 'n',
  15: 'o',
  16: 'p',
  17: 'q',
  18: 'r',
  19: 's',
  20: 't',
  21: 'u',
  22: 'v',
  23: 'w',
  24: 'x',
  25: 'y',
  26: 'z',
  0: '.'})

In [14]:
X, Y =[],[]
context_length = 3

for w in words[:3]:
    context_block = [0] * context_length
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context_block)
        Y.append(ix)
        print(''.join(itos[i]for i in context_block),"-->",itos[ix])
        context_block = context_block[1:] + [ix]

... --> e
..e --> m
.em --> m
emm --> a
mma --> .
... --> o
..o --> l
.ol --> i
oli --> v
liv --> i
ivi --> a
via --> .
... --> a
..a --> v
.av --> a
ava --> .


In [18]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [19]:
X.shape,Y.shape

(torch.Size([16, 3]), torch.Size([16]))

In [20]:
C = torch.rand((27,2))

In [21]:
emb = C[X]
emb.shape

torch.Size([16, 3, 2])

In [23]:
import torch.nn.functional as F

In [24]:
C = torch.rand((27,2))
W1 = torch.rand(6,100)
b1 = torch.rand(100)
W2 = torch.rand(100,27)
b2 = torch.rand(27)
parameters = [C,W1,b1,W2,b2]

In [25]:
for p in parameters:
    p.requires_grad=True

In [28]:
for _ in range(100):
    emb=C[X]
    h = torch.tanh(emb.view(-1,6) @ W1+b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Y)

    for p in parameters:
        p.grad = None
    loss.backward()

    for p in parameters:
        p.data += -0.1 * p.grad
    print(loss.item())

0.7294543981552124
0.7232506275177002
0.7171263098716736
0.7110769152641296
0.7050917744636536
0.6991667747497559
0.6932919025421143
0.687462568283081
0.6816680431365967
0.675906777381897
0.6701691746711731
0.6644508838653564
0.6587471961975098
0.6530494093894958
0.6473528146743774
0.64165198802948
0.6359444856643677
0.6302292346954346
0.624496340751648
0.6187463402748108
0.6129797101020813
0.6071922779083252
0.6013861298561096
0.5955617427825928
0.5897229313850403
0.5838703513145447
0.5780113935470581
0.5721516609191895
0.5662981271743774
0.5604606866836548
0.5546465516090393
0.5488659739494324
0.5431289672851562
0.5374462604522705
0.5318260788917542
0.5262792706489563
0.5208114385604858
0.5154327154159546
0.5101445317268372
0.5049542784690857
0.4998665750026703
0.49488189816474915
0.4900028705596924
0.4852283000946045
0.4805558919906616
0.4759873151779175
0.4715161621570587
0.467144638299942
0.46286624670028687
0.458678662776947
0.4545782804489136
0.4505641460418701
0.446631580591201

In [30]:
block_size=3

In [32]:
for _ in range(20):
    out =[]
    context = [0]* block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1,-1)@ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits,dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
    print(''.join(itos[i] for i in out))

emma.
emma.
oliviaa.
ava.
av.
evia.
emma.
emmv.
emma.
avaa.
ava.
emma.
av.
ava.
olivia.
ava.
emma.
emmv.
olivi.
emma.
